# Librerias

In [12]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

In [13]:
from imblearn.over_sampling import SMOTE

# Modalamiento

In [3]:
# 1. Cargar los datasets
train = pd.read_csv("TrainingDataset/train_history.csv", parse_dates=["date"])
test = pd.read_csv("TrainingDataset/test_history.csv", parse_dates=["date"])

# 2. Definir features y target
features = ["price", "ma7", "rsi14"]  # se puede añadir más 
target = "target_up_30d"

## Entrenar el modelo y evaluacion

In [4]:
X_train = train[features]
y_train = train[target]

X_test = test[features]
y_test = test[target]

# 3. Entrenar modelo
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 4. Predicción y evaluación
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]


## Reportes de Clasificacion

In [5]:
print("Classification Report:")
print(classification_report(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.97      0.93     58230
           1       0.16      0.05      0.08      7230

    accuracy                           0.87     65460
   macro avg       0.53      0.51      0.50     65460
weighted avg       0.81      0.87      0.83     65460



| Clase | Descripción             | Precision | Recall | F1-score | Support |
| ----- | ----------------------- | --------- | ------ | -------- | ------- |
| 0     | No sube ≥20% en 30 días | 0.89      | 0.97   | 0.93     | 58,230  |
| 1     | Sí sube ≥20% en 30 días | 0.16      | 0.05   | 0.08     | 7,230   |


| Métrica       | Clase 0 (No sube ≥20%) | Clase 1 (Sí sube ≥20%) | Interpretación                                                                                                                                  |
| ------------- | ---------------------- | ---------------------- | ----------------------------------------------------------------------------------------------------------------------------------------------- |
| **Precision** | 0.89                   | 0.16                   | De todos los que el modelo predijo como 0 o 1, estos fueron los correctos. Para clase 1, solo el 16% de las predicciones fueron correctas.      |
| **Recall**    | 0.97                   | 0.05                   | El 97% de los reales clase 0 fueron detectados, pero solo el 5% de los reales clase 1 se detectaron. El modelo ignora casi todos los positivos. |
| **F1-score**  | 0.93                   | 0.08                   | F1 combina precisión y recall. La clase 1 tiene un desempeño pésimo: el modelo falla tanto en detectar como en acertar.                         |
| **Support**   | 58,230                 | 7,230                  | Tamaño real de cada clase en los datos. La clase 0 domina (alta desbalance).                                                                    |


🚨 Conclusión

El modelo aprendió a predecir solo la clase mayoritaria (0 = no sube) y ignora los casos importantes (1 = sí sube ≥20%).

In [6]:
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Confusion Matrix:
[[56268  1962]
 [ 6857   373]]


|                            | Predicho: 0       | Predicho: 1    | Total Real |
| -------------------------- | ----------------- | -------------- | ---------- |
| **Real: 0** (no sube ≥20%) | **56,268** (✅ TN) | 1,962 (❌ FP)   | 58,230     |
| **Real: 1** (sí sube ≥20%) | 6,857 (❌ FN)      | **373** (✅ TP) | 7,230      |


🧠 Interpretación de cada celda

- TN (56,268): Casos correctamente predichos como clase 0 (no sube).

- FP (1,962): Casos que no subían pero el modelo dijo que sí (falsos positivos).

- FN (6,857): Casos que sí subían pero el modelo no los detectó (falsos negativos). ⚠️

- TP (373): Casos correctamente detectados como subida ≥20% (positivos reales).

🚨 Conclusión:

El modelo predice muy bien la clase 0, pero casi ignora la clase 1.

La mayoría de las subidas reales no fueron detectadas (solo 373 de 7,230 → 5.1% recall).

Muchos falsos negativos: si tu objetivo es identificar subidas, este modelo aún no es útil para acción.

In [7]:
print("\nROC AUC:")
print(roc_auc_score(y_test, y_proba))


ROC AUC:
0.5433011185908695


| Valor AUC | Interpretación                                                                                                                                              |
| --------- | ----------------------------------------------------------------------------------------------------------------------------------------------------------- |
| 0.543     | Muy bajo. El modelo **apenas supera el azar** en distinguir subidas reales. Es señal de que **no está aprendiendo patrones útiles** sobre cuándo sube ≥20%. |


⚠️ Conclusión:

Aunque el accuracy global es 87%, el AUC muestra que no está diferenciando bien entre subir/no subir.

Esto se debe al desbalance de clases (muchas más clases 0 que 1).

## Entrenamiento con class_weight y Predicciones

In [8]:
# 1. Entrenamiento con class_weight
clf = RandomForestClassifier(
    class_weight='balanced',  # 💡 esta es la clave
    random_state=42,
    n_jobs=-1
)
clf.fit(X_train, y_train)

# 2. Predicciones
y_pred = clf.predict(X_test)
y_proba = clf.predict_proba(X_test)[:, 1]

## Reporte de métricas

In [9]:

print("Classification Report:\n", classification_report(y_test, y_pred))

Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.97      0.93     58230
           1       0.17      0.05      0.08      7230

    accuracy                           0.87     65460
   macro avg       0.53      0.51      0.50     65460
weighted avg       0.81      0.87      0.84     65460



In [10]:
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


Confusion Matrix:
 [[56469  1761]
 [ 6866   364]]


In [11]:
print("ROC AUC:\n", roc_auc_score(y_test, y_proba))

ROC AUC:
 0.540566140993328


## SMOTE + Random Forest

### verificar y limpiar los NaN en X_train

In [15]:
# Ver cuántos nulos hay en X_train
print(X_train.isnull().sum())

price        0
ma7       7119
rsi14    15386
dtype: int64


In [16]:
# Opción rápida: eliminar filas con NaN
X_train_clean = X_train.dropna()
y_train_clean = y_train.loc[X_train_clean.index]

### Aplicar SMOTE

In [18]:
# 1. Aplica SMOTE al conjunto de entrenamiento
smote = SMOTE(random_state=42)
X_train_bal, y_train_bal = smote.fit_resample(X_train_clean, y_train_clean)

# 2. Entrena el modelo
clf = RandomForestClassifier(
    random_state=42,
    n_jobs=-1
)
clf.fit(X_train_bal, y_train_bal)

# 3. Predicciones
y_pred = clf.predict(X_test)
y_proba = clf.predict_proba(X_test)[:, 1]

### Reportes

In [19]:
# Reporte de métricas
print("📌 Classification Report:")
print(classification_report(y_test, y_pred))

📌 Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.60      0.72     58230
           1       0.12      0.44      0.19      7230

    accuracy                           0.58     65460
   macro avg       0.51      0.52      0.45     65460
weighted avg       0.81      0.58      0.66     65460



| Clase             | Precision | Recall | F1-score | Soporte | Interpretación                                                                                                                                          |
| ----------------- | --------- | ------ | -------- | ------- | ------------------------------------------------------------------------------------------------------------------------------------------------------- |
| 0 (no subida)     | 0.90      | 0.60   | 0.72     | 58,230  | **Muy precisa**, pero ahora **se confunde más** que antes (baja el recall de 0.97 a 0.60). Detecta bien las negativas pero deja pasar muchas positivas. |
| 1 (subida >= 20%) | 0.12      | 0.44   | 0.19     | 7,230   | **Baja precisión**, pero **recall mejoró mucho** (de 0.05 a 0.44). El modelo ahora detecta más clases 1 aunque muchas son falsos positivos.             |
| **Accuracy**      | -         | -      | **0.58** | -       | Bajó bastante, pero es esperable: **hay más equilibrio entre clases ahora**.                                                                            |
| **Macro avg**     | 0.51      | 0.52   | 0.45     | -       | Mejor que antes (0.53/0.51/0.50), refleja que **ambas clases están más equilibradas**.                                                                  |
| **Weighted avg**  | 0.81      | 0.58   | 0.66     | -       | Menor que antes por caída en clase 0, pero más justo.                                                                                                   |


In [20]:
# Matriz de confusión
print("📌 Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

📌 Confusion Matrix:
[[34823 23407]
 [ 4014  3216]]


| Métrica                  | Valor  | Explicación                                                                    |
| ------------------------ | ------ | ------------------------------------------------------------------------------ |
| **True Positives (TP)**  | 3,216  | Casos de clase 1 correctamente predichos como 1 (subidas detectadas).          |
| **False Positives (FP)** | 23,407 | Casos de clase 0 que fueron incorrectamente predichos como 1 (falsas alarmas). |
| **True Negatives (TN)**  | 34,823 | Casos de clase 0 correctamente predichos como 0.                               |
| **False Negatives (FN)** | 4,014  | Casos de clase 1 que no se detectaron (falsos negativos).                      |


✅ Conclusiones:

- El modelo detecta 3,216 subidas reales (TP), mucho mejor que antes (solo 373 con class_weight).

- Pero también se equivoca más prediciendo subidas cuando no las hay (23,407 FP).

- Este es el efecto esperado del SMOTE: más sensibilidad, menos precisión.

- Esto mejora el recall de la clase 1 (44%) a cambio de bajar el recall de la clase 0 (60%).

In [21]:
# AUC-ROC
print("📌 ROC AUC Score:")
print(roc_auc_score(y_test, y_proba))

📌 ROC AUC Score:
0.5386167933760077


| Métrica           | Valor    | Interpretación                                    |
| ----------------- | -------- | ------------------------------------------------- |
| **AUC-ROC Score** | `0.5386` | El modelo apenas supera el azar (que sería 0.50). |


📉 ¿Qué significa esto?

El valor 0.5386 indica que el modelo tiene poca capacidad para distinguir entre clases 0 y 1.

En un problema desbalanceado, es común que un modelo parezca “bueno” por accuracy, pero su AUC revela que casi no diferencia entre positivos y negativos reales.

En este caso, incluso con SMOTE, el modelo aumentó recall pero no mejoró sustancialmente su capacidad de discriminación.

In [24]:
import pandas as pd

# 1. Carga el dataset original
df = pd.read_csv("TrainingDataset/training_from_history.csv")

# 2. Asegúrate de usar las mismas features que en el entrenamiento
features = ['price', 'ma7', 'rsi14']
X = df[features].copy()

# 3. Elimina filas con nulos en las features si es necesario
X = X.dropna()
df = df.loc[X.index]  # sincroniza índices

# 4. Genera las predicciones de probabilidad
df['proba_up_30d'] = clf.predict_proba(X)[:, 1]

# 5. Prepara columnas necesarias para la app
df['proba_up_60d'] = df['proba_up_30d']  # si aún no entrenas otros modelos
df['proba_up_90d'] = df['proba_up_30d']

cols_finales = [
    'symbol', 'price',
    'proba_up_30d', 'target_return_30d',
    'proba_up_60d', 'target_return_60d',
    'proba_up_90d', 'target_return_90d'
]

df_app = df[cols_finales]
df_app.to_csv("data/predicciones.csv", index=False)
print("✅ Archivo predicciones.csv guardado.")


✅ Archivo predicciones.csv guardado.


In [26]:
import pandas as pd

# Cargar los symbols únicos del dataset
df_narr = pd.read_csv("data/narrativas.csv")
df_narr['symbol'] = df_narr['symbol'].str.replace('$', '', regex=False)

symbols_unicos = df['symbol'].unique()

# Crear DataFrame base de narrativas
df_narr = pd.DataFrame({
    'symbol': symbols_unicos,
    'narrativa': ''  # ← Aquí tú completas manualmente
})

# Guardar CSV para completar
df_narr.to_csv("data/narrativas.csv", index=False)
print("✅ Archivo 'narrativas.csv' generado. Rellénalo con IA, Videojuegos, RWA o Memes.")


✅ Archivo 'narrativas.csv' generado. Rellénalo con IA, Videojuegos, RWA o Memes.
